In [1]:
import stim
import random
import time
import os
import math
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator


import stim

folder_path = "/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/stim_to_qasm_files"
# Ensure the folder exists
if not os.path.isdir(folder_path):
    print(f"The folder '{folder_path}' does not exist.")

# List all files in the folder
qasm_files = [f for f in os.listdir(folder_path) if f.endswith('.qasm')]

if not qasm_files:
    print("No .qasm files found in the folder.")

for qasm_file in qasm_files:
    full_path = os.path.join(folder_path, qasm_file)

    qc = QuantumCircuit.from_qasm_file(full_path)

    sim = AerSimulator(method="stabilizer")
    n_qubits = qc.num_qubits

    if n_qubits < 7000:
        start = time.perf_counter()
        result = sim.run(qc).result()
        end = time.perf_counter()
        elapsed = end - start

        output_dir = "/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/surface_operation_bench"

        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.join(output_dir, f"qiskit_{n_qubits}.txt")
        with open(filename, "w") as f:
            f.write("qiskit\n")
            f.write(f"{elapsed}\n")
            f.write(f"{n_qubits}\n")
    
    print("qiskit done\n")


qiskit done

circuit done

R 103 105 107 109 111 113 115 117 119 121 123 203 205 207 209 211 213 215 217 219 221 223 303 305 307 309 311 313 315 317 319 321 323 403 405 407 409 411 413 415 417 419 421 423 503 505 507 509 511 513 515 517 519 521 523 603 605 607 609 611 613 615 617 619 621 623 703 705 707 709 711 713 715 717 719 721 723 803 805 807 809 811 813 815 817 819 821 823 903 905 907 909 911 913 915 917 919 921 923 1003 1005 1007 1009 1011 1013 1015 1017 1019 1021 1023 1103 1105 1107 1109 1111 1113 1115 1117 1119 1121 1123 5603 5605 5607 5609 5611 5613 5615 5617 5619 5621 5623 5703 5705 5707 5709 5711 5713 5715 5717 5719 5721 5723 5803 5805 5807 5809 5811 5813 5815 5817 5819 5821 5823 5903 5905 5907 5909 5911 5913 5915 5917 5919 5921 5923 6003 6005 6007 6009 6011 6013 6015 6017 6019 6021 6023 6103 6105 6107 6109 6111 6113 6115 6117 6119 6121 6123 6203 6205 6207 6209 6211 6213 6215 6217 6219 6221 6223 6303 6305 6307 6309 6311 6313 6315 6317 6319 6321 6323 6403 6405 6407 6409 6411 

KeyboardInterrupt: 

In [ ]:
import stim

folder_path = "/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/stim_to_stim_files"
# Ensure the folder exists
if not os.path.isdir(folder_path):
    print(f"The folder '{folder_path}' does not exist.")

# List all files in the folder
stim_files = [f for f in os.listdir(folder_path) if f.endswith('.stim')]

if not stim_files:
    print("No .stim files found in the folder.")

for stim_file in stim_files:
    circuit = stim.Circuit()

    # Iterate over Qiskit's circuit data using the recommended access pattern
    for instruction in qc.data:
        inst = instruction.operation
        qargs = instruction.qubits
        # clbits = instruction.clbits  # You can include this if you need classical bits

        # Find the qubit indices
        qargs_indices = [qc.qubits.index(qubit) for qubit in qargs]

        # Map Qiskit gates to Stim gates
        if inst.name == 'h':
            circuit.append("H", qargs_indices)
        elif inst.name == 'cx':
            for i in range(0, len(qargs_indices), 2):
                circuit.append("CX", [qargs_indices[i], qargs_indices[i+1]])
        elif inst.name == 's':
            circuit.append("S", qargs_indices)
        elif inst.name == 'reset':
            circuit.append("R", qargs_indices)
        elif inst.name == 'measure':
            circuit.append("M", qargs_indices)
        elif inst.name in {'barrier', 'id'}:
            continue
        else:
            raise NotImplementedError(f"Gate '{inst.name}' is not supported by Stim")

    print("circuit done\n")
    print(circuit)
    simulator = stim.TableauSimulator()

    
    start = time.perf_counter()
    simulator.do_circuit(circuit)
    end = time.perf_counter()
    elapsed = end - start

    os.makedirs(output_dir, exist_ok=True)
    filename = os.path.join(output_dir, f"stim_files_{n_qubits}.txt")
    with open(filename, "w") as f:
        f.write("stim\n")
        f.write(f"{elapsed}\n")
        f.write(f"{n_qubits}\n")
    
    print("stim done\n")
